In [21]:
#import statements
import numpy as np
import pandas as pd
import operator as op

from pyspark.sql import HiveContext, SparkSession
hive_context = HiveContext(sc)


from pyspark.sql.functions import col, lit
from pyspark.sql.types import *
from pyspark.ml.linalg import *
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, HashingTF, Tokenizer, StringIndexer, CountVectorizer, IDF, Word2Vec
from pyspark.ml import Pipeline

In [22]:
spark = SparkSession.builder.enableHiveSupport().appName('ReadWriteData').getOrCreate()
sc = spark.sparkContext

In [23]:
# connect to HDFS
!hdfs dfs -ls /user/vvenkatesan/final_project/

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0
Found 9 items
drwxr-xr-x   - vvenkatesan vvenkatesan          0 2020-08-06 12:47 /user/vvenkatesan/final_project/bus_cat
drwxr-xr-x   - kleindiek   vvenkatesan          0 2020-08-02 12:03 /user/vvenkatesan/final_project/business_json
-rw-r--r--   3 kleindiek   vvenkatesan  248796774 2020-08-04 14:56 /user/vvenkatesan/final_project/review_subset.json
-rw-r--r--   3 vvenkatesan vvenkatesan  449663480 2020-07-09 18:53 /user/vvenkatesan/final_project/yelp_academic_dataset_checkin.json
-rw-r--r--   3 vvenkatesan vvenkatesan 6325565224 2020-07-09 18:54 /user/vvenkatesan/final_project/yelp_academic_dataset_review.json
-rw-r--r--   3 kleindiek   vvenkatesan  175454655 2020-07-28 17:37 /user/vvenkatesan/final_project/yelp_academic_dataset_tip.csv
-rw-r--r--   3 vvenkatesan vvenkatesan  263489322 2020-07-09 18:55 /user/vvenkatesan/final_project/yelp_academic_dataset_tip.json
-rw-r--r--   3 tia

## Content-Based Recommendation Engine

### 1) Loading Business and Review Data

In [24]:
df_load = spark.read.csv('/user/vvenkatesan/final_project/bus_cat/')

In [25]:
df_load.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)



In [26]:
df_load.show(5)

+--------------------+----------------+--------------------+
|                 _c0|             _c1|                 _c2|
+--------------------+----------------+--------------------+
|f9NumwFMBDn751xgF...|     Active Life|    Gun/Rifle Ranges|
|Yzvjg0SayhoZgCljU...|Health & Medical|Fitness & Instruc...|
|XNoUzKckATkOD1hP6...|            Pets|        Pet Services|
|6OAZjbxqM5ol29BuH...| Hardware Stores|       Home Services|
|51M2Kk903DFYI6gnB...|   Home Services|            Plumbing|
+--------------------+----------------+--------------------+
only showing top 5 rows



In [27]:
bus_cat = df_load.select(col("_c0").alias("business_id"), col("_c1").alias("cat_primary"), col("_c2").alias("cat_secondary"))
bus_cat.show(5)

+--------------------+----------------+--------------------+
|         business_id|     cat_primary|       cat_secondary|
+--------------------+----------------+--------------------+
|f9NumwFMBDn751xgF...|     Active Life|    Gun/Rifle Ranges|
|Yzvjg0SayhoZgCljU...|Health & Medical|Fitness & Instruc...|
|XNoUzKckATkOD1hP6...|            Pets|        Pet Services|
|6OAZjbxqM5ol29BuH...| Hardware Stores|       Home Services|
|51M2Kk903DFYI6gnB...|   Home Services|            Plumbing|
+--------------------+----------------+--------------------+
only showing top 5 rows



In [76]:
bus = spark.read.json("/user/vvenkatesan/final_project/business_json/yelp_academic_dataset_business.json")
bus.printSchema()

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [77]:
bus = bus.drop("count", "state", "postal_code", "latitude", "longitude", "attributes", "categories", "hours")
bus = bus.filter(bus.business_id != 'business_id')

bus = bus.join(bus_cat, on = ['business_id'], how = 'left')
bus = bus.drop('cat_secondary')
bus.show(5)

+--------------------+--------------------+---------------+-------+--------------------+------------+-----+----------------+
|         business_id|             address|           city|is_open|                name|review_count|stars|     cat_primary|
+--------------------+--------------------+---------------+-------+--------------------+------------+-----+----------------+
|f9NumwFMBDn751xgF...|     10913 Bailey Rd|      Cornelius|      1|The Range At Lake...|          36|  3.5|     Active Life|
|Yzvjg0SayhoZgCljU...|8880 E Via Linda,...|     Scottsdale|      1|   Carlos Santo, NMD|           4|  5.0|Health & Medical|
|XNoUzKckATkOD1hP6...|3554 Rue Notre-Da...|       Montreal|      1|             Felinus|           5|  5.0|            Pets|
|6OAZjbxqM5ol29BuH...|      1015 Sharp Cir|North Las Vegas|      0|Nevada House of Hose|           3|  2.5| Hardware Stores|
|51M2Kk903DFYI6gnB...|  4827 E Downing Cir|           Mesa|      1|USE MY GUY SERVIC...|          26|  4.5|   Home Services|


In [29]:
review = spark.sql('SELECT * FROM big_data_group_2.review')
review.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: integer (nullable = true)
 |-- funny: integer (nullable = true)
 |-- cool: integer (nullable = true)



In [30]:
review = review.drop('date', 'useful', 'funny', 'cool')
review.show(5)

+--------------------+--------------------+--------------------+-----+--------------------+
|           review_id|             user_id|         business_id|stars|                text|
+--------------------+--------------------+--------------------+-----+--------------------+
|xQY8N_XvtGbearJ5X...|OwjRMXRC0KyPrIlcj...|-MhfebM0QIsKt87iD...|  2.0|As someone who ha...|
|UmFMZ8PyXZTY2Qcwz...|nIJD_7ZXHq-FX8byP...|lbrU8StCq3yDfr-QM...|  1.0|I am actually hor...|
|LG2ZaYiOgpr2DK_90...|V34qejxNsCbcgD8C0...|HQl28KMwrEKHqhFrr...|  5.0|I love Deagan's. ...|
|i6g_oA9Yf9Y31qt0w...|ofKDkJKXSKZXu5xJN...|5JxlZaqCnk1MnbgRi...|  1.0|Dismal, lukewarm,...|
|6TdNDKywdbjoTkize...|UgMW8bLE0QMJDCkQ1...|IS4cv902ykd8wj1TR...|  4.0|Oh happy day, fin...|
+--------------------+--------------------+--------------------+-----+--------------------+
only showing top 5 rows



In [71]:
review.count()

80211220

#### Build a subset of the data!

In [32]:
review_sub = review.rdd.takeSample(False, 100000, seed = 27)
review_sub = spark.createDataFrame(review_sub)
review_sub.count()

100000

In [33]:
review_sub.show(5)

+--------------------+--------------------+--------------------+-----+--------------------+
|           review_id|             user_id|         business_id|stars|                text|
+--------------------+--------------------+--------------------+-----+--------------------+
|HS0F54bYyeX1tW6Sw...|SwXA_s37pFvgqGDfn...|pybIuTluqRhH_BQ3C...|  5.0|my sister was maj...|
|rXs-HVqRQdkpnFJA2...|wfumINHN_Vi3Ntf2M...|KskYqH1Bi7Z_61pH6...|  5.0|I lived in Thaila...|
|6eCfo4OjwWqMoFHG1...|8qnHcAlkimpch0RTT...|szL4LjaGZpmsLjQ5A...|  1.0|This place is a j...|
|RJbGFZG-WiqOhQr1x...|W_iZ-l3zOg4Gg4Nr4...|BIuXMWuQIlC1ZlC3l...|  5.0|I absolutely love...|
|__MtCRuidIjS83Uib...|ZOB5CrdUEOM0qKGLM...|vHz2RLtfUMVRPFmd7...|  5.0|This is a must tr...|
+--------------------+--------------------+--------------------+-----+--------------------+
only showing top 5 rows



In [34]:
# select just the business id and the text of the review

subset_text_only = review_sub.select(col("business_id"), col("text"))
subset_text_only.show(5)

+--------------------+--------------------+
|         business_id|                text|
+--------------------+--------------------+
|pybIuTluqRhH_BQ3C...|my sister was maj...|
|KskYqH1Bi7Z_61pH6...|I lived in Thaila...|
|szL4LjaGZpmsLjQ5A...|This place is a j...|
|BIuXMWuQIlC1ZlC3l...|I absolutely love...|
|vHz2RLtfUMVRPFmd7...|This is a must tr...|
+--------------------+--------------------+
only showing top 5 rows



In [35]:
# concatenate the review text -- we want to look at all reviews for each business!

# concatentate based on business ID as key. must be in tuple form
subset_conc = subset_text_only.rdd.map(tuple).reduceByKey(op.add)

# create a dataframe of the results to use for tokenization / featurization!
subset_conc_df = spark.createDataFrame(subset_conc).withColumnRenamed('_1', 'business_id').withColumnRenamed('_2', 'text_from_reviews')
subset_conc_df.count()

47267

### 2) Tokenization / Featurization

In [36]:
# build a pipeline

# make each word a token
tokenizer = RegexTokenizer(gaps = False, pattern = '\w+', inputCol = 'text_from_reviews', outputCol = 'tokenized_text')

# remove stopwords like the, etc
stopwords_remove = StopWordsRemover(inputCol = 'tokenized_text', outputCol = 'tokenized_nostopwords')

# vectorize words
word_2_vec = Word2Vec(vectorSize = 100, minCount = 5, inputCol = "tokenized_nostopwords", outputCol = "wordVectors")

pipeline = Pipeline(stages = [tokenizer, stopwords_remove, word_2_vec])

In [37]:
# fit model pipeline
model = pipeline.fit(subset_conc_df)

# transform after fitting
review_subset_tansformed = model.transform(subset_conc_df)
review_subset_tansformed.show(2)

In [38]:
# visualize the calculated vectors. we don't care about intermediate columns anymore and we need to see at least one business
# id for the test case

reviews_vectors = review_subset_tansformed.select('business_id', 'wordVectors')
reviews_vectors = reviews_vectors.rdd.map(lambda x: (x[0], x[1])).collect()
reviews_vectors[0:5]

[('OWll9a5nBADV25pAFh7wPw',
  DenseVector([-0.0376, -0.0021, -0.0663, -0.0572, 0.0286, 0.0101, 0.0144, 0.0237, -0.0879, 0.0459, 0.0436, -0.0711, -0.0245, -0.0038, -0.0499, 0.0472, 0.0268, 0.0028, 0.0417, 0.068, 0.0296, 0.0119, 0.0292, 0.0818, 0.0126, 0.0178, -0.0111, 0.0489, 0.0255, -0.0029, 0.0242, 0.0026, -0.0416, 0.0617, 0.067, 0.0165, -0.0819, -0.0118, 0.0158, 0.0221, 0.1095, 0.0423, -0.0008, -0.0615, 0.0191, 0.0444, -0.0042, 0.0077, 0.0473, -0.0415, 0.0174, 0.0515, -0.0271, -0.0687, 0.0181, 0.042, -0.0526, 0.015, -0.0812, 0.0405, -0.0373, -0.0153, -0.0311, 0.0286, -0.0525, 0.0523, 0.0244, -0.0265, -0.015, -0.0342, -0.0965, 0.0098, -0.0467, -0.0928, 0.019, -0.0297, 0.0316, 0.0811, 0.0494, 0.0363, -0.045, 0.0047, -0.0296, 0.0486, -0.0524, -0.0536, -0.0045, -0.0355, -0.0259, 0.0304, 0.0482, -0.0319, 0.0692, 0.0988, -0.0634, -0.0565, 0.0119, 0.0126, 0.0935, -0.0137])),
 ('q18xbq3Cbyp_BJyfMQxFxg',
  DenseVector([-0.0419, 0.0043, -0.0428, -0.1367, 0.0292, -0.0057, 0.0451, 0.0614, -0.044

In [104]:
reviews_vectors[11:20]

[('PMcrF8hMxaPklx-Yf5mWkA',
  DenseVector([0.0261, 0.0295, -0.0788, -0.109, -0.0146, -0.0524, 0.0406, 0.0811, -0.0922, 0.0456, 0.0545, -0.0516, 0.0427, -0.0177, -0.0501, -0.0289, 0.0661, 0.0625, 0.0298, 0.0578, 0.0256, 0.021, 0.0198, 0.0892, 0.0987, 0.0144, -0.0278, -0.0186, 0.0308, -0.0176, 0.1, 0.066, -0.0444, 0.001, 0.1121, -0.0049, -0.0519, -0.0544, 0.0631, -0.0056, 0.082, 0.0847, -0.0174, -0.1175, -0.0258, 0.0309, 0.0458, 0.0724, 0.1354, 0.0152, -0.0117, 0.0035, -0.0218, 0.0331, 0.0617, 0.0469, 0.0271, 0.0313, -0.0268, 0.0445, -0.056, 0.0166, 0.0399, 0.0729, -0.0505, 0.0314, 0.0158, 0.0626, 0.0146, -0.003, -0.0119, -0.0331, 0.0008, -0.0858, -0.0064, -0.0363, -0.0122, 0.0069, 0.0909, -0.0496, -0.0334, -0.0, -0.0847, -0.0007, -0.0018, -0.0601, 0.0074, 0.0447, -0.0688, 0.0093, 0.0711, -0.0624, 0.007, 0.0365, -0.0727, -0.0478, -0.0974, -0.0999, 0.1097, -0.0149])),
 ('Q4AWWdBWe_ojYX1a4ffXFQ',
  DenseVector([-0.0763, -0.0112, -0.0088, -0.0981, -0.0001, 0.0857, 0.0371, 0.0401, -0.0884, 0

### 3) Define Recommendation Functions

We need to be able to identify similar business and reviews. What types of content based recommendation engines can we build?

1. similar business
2. recommend based on what user reviewed perviously
3. recommendation based on a key word search

In [78]:
# define cosine similarity. Cosine similarity is a measurement used to gauge how comparative the records 
# are independent of their size. Computes cosine of the edge between two vectors anticipated in a multi-dimensional space.

def CosineSimilarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [79]:
# define function to compute the cosine similarity of the businesses in our data

def BusinessScore(bus_name):
    
    business_id = (bus.select("business_id").filter(col("name") == lit(bus_name))).head()[0]
    
    print('-----Compiling Word Vectors-----')
    
    bus_vector = (review_subset_tansformed.select('wordVectors')
                  .filter(review_subset_tansformed['business_id'] == business_id).collect()[0][0])
    
    print('-----Computing Cosine Similarity-----')
    
    bus_vector_rdd = sc.parallelize((i[0], float(CosineSimilarity(bus_vector, i[1]))) for i in reviews_vectors)
    
    # build a dataframe with business ID and cosine similarity as 'score'
    bus_score = spark.createDataFrame(bus_vector_rdd)
    bus_score = (bus_score.withColumnRenamed('_1', 'business_id').withColumnRenamed('_2', 'score')
                               .orderBy(['score'], ascending=[0]))
    
    # we don't want the business id that we're searching to be in the results or NAs either
    bus_score = bus_score.filter(col("business_id") != business_id).filter(col("score") != 'NaN')
    
    print('-----Similar Businesses Found-----')
                
    return bus_score

In [80]:
def BusinessInformation(bus_score):
    
    print('-----Collecting Business Information-----')
    
    selected_ids = bus_score.select('business_id').rdd.flatMap(lambda x: x).collect()
    
    business_subset = bus.where(bus.business_id.isin(selected_ids))
    business_subset = business_subset.join(bus_score, on = ['business_id'], how = 'left')
    
    print('-----Removing Closed Businesses-----')
    
    business_subset = business_subset.filter('is_open == 1')
    business_subset = business_subset.drop('is_open')
    
    print('-----Removing Businesses With Worse Than 3.5 Rating-----')
    
    business_subset = business_subset.filter('stars > 3.0')
    
    print('-----Filtering Based On Category and Location-----')
    
    category = (bus.select("cat_primary").filter(col("name") == lit(bus_name))).head()[0]
    city = (bus.select("city").filter(col("name") == lit(bus_name))).head()[0]
    
    print('-----Compiling Final Recommendations-----')
    
    trial = (business_subset.select('name', 'address','city','cat_primary','score').filter(business_subset.city == city)
             .filter(business_subset.cat_primary == category).sort("score").orderBy(["score"], ascending=[0])).limit(5)
    
    return trial.toPandas()
    

In [108]:
def InputBusinessInformation(bus_name):
    
    print('-----Pulling Initial Business Information-----')
    
    bus_info = (bus.select('name', 'address','city','cat_primary').filter(bus.name == bus_name)).limit(1)
    bus_info = bus_info.toPandas()
       
    return bus_info

### 4) Test Case!

In [114]:
bus_name = "Bouchon Bakery"

test = BusinessInformation(BusinessScore(bus_name))
input_info = InputBusinessInformation(bus_name)

display(input_info, test)

-----Compiling Word Vectors-----
-----Computing Cosine Similarity-----


/software/Anaconda3-5.1.0-hadoop/lib/python3.6/site-packages/ipykernel/__main__.py:5: RuntimeWarning: invalid value encountered in double_scalars


-----Similar Businesses Found-----
-----Collecting Business Information-----
-----Removing Closed Businesses-----
-----Removing Businesses With Worse Than 3.5 Rating-----
-----Filtering Based On Category and Location-----
-----Compiling Final Recommendations-----
-----Pulling Initial Business Information-----


,name,address,city,cat_primary
0,Bouchon Bakery,"3355 Las Vegas Blvd S, Venetian Theater, Sport...",Las Vegas,Food


,name,address,city,cat_primary,score
0,Pinkbox Doughnuts,7531 W Lake Mead Blvd,Las Vegas,Food,0.907878
1,Drago Sisters Bakery,"6870 S Rainbow Blvd, Ste 116",Las Vegas,Food,0.877925
2,District: Donuts. Sliders. Brew,"3708 Las Vegas Blvd S, Fl 2, The Blvd Tower",Las Vegas,Food,0.874499
3,Nielsen’s Frozen Custard,"9480 S Eastern Ave, Ste 100",Las Vegas,Food,0.862000
4,Honolulu Cookie Company - Grand Canal Shoppes,3327 Las Vegas Blvd S,Las Vegas,Food,0.859122
